# Seminar - JSON, XML, Requests, Web-Scraping 
by Vítek Macháček

## Task 1: JSON API and Quandl

Set Quandl access credentials

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

QUANDL_API_KEY = 'YOUR-QUANDL-API-KEY'


Request Quandl for time series:

hint: The most simple Quandl Format for API-call is:

https://www.quandl.com/api/v3/datasets/{QUANDL_DATABASE}/{QUANDL_DATASET}/data.{DATA_FORMAT}?api_key={YOUR-QUANDL-API-KEY}

Let's ask for `FB` dataset from `WIKI` database. The results should be in JSON format

convert response to python natives

What keys does a dictionary contain?

Let's go deeper

We want to see the data

first data point?

Let's do a DataFrame

Perhaps column names can be useful

## Task 1b: XML (not covered on seminar)
Request XML format of the same dataset

Let just save a XML into a file `xml_sample.xml`

get BeatifulSoup object

OK gimme a dataframe

## Task 2: Quandl website

Let's get a list of available datasets on Quandl:

The list of datasets on Quandl website is available at: https://www.quandl.com/search?filters=%5B%22Free%22%5D

Is this website scrapable? 

## Task 3: Scraping IES news

use following code snippets to construct your own IES News web scraper

In [ ]:
def get_soup(link):
    '''
    Function accepts a link and returns a BeautifulSoup object parsed from text of a succesful GET request on a link. If requests returns other status code than 200, returns None and prints a message

    Make sure that the request object is parsed as UTF-8 string.
    '''
    pass

In [ ]:
def get_all_news_links(url):
    '''
    Generates list of URLs of all news-related links from the url provided.

    Links on news format: <a href="/en/news/{id}" title="show news" class="show-news">show news</a>

    The URLs are expected in absolute format, i.e. including a full domain.
    '''
    pass

In [ ]:
def parse_title(soup):
    '''
    Parse text of the first `h3` object from the soup element.
    '''
    pass

In [ ]:
def parse_date(soup):
    '''
    Parse text of the sibling of sibling of the first h3 element in the soup. Note that the immidiate sibling of `h3` is not Tag element, but NavigableString. This is used to represent text between tags.
    '''

    pass

In [ ]:
def parse_news_content(soup):
    '''
    For simplicity, the content of the article is the content of all <p> elements within <div class="col-sm-12 news"></>

    Return a single string with the whole text. Use `/n` as a connecting string between individual p-texts. 

    Hint: Consider using a `.join()` function applicable on string object
    '''
    pass

In [ ]:
def parse_ies_news(link,pause=.5):
    '''
    From URL of given news story generate pd.Series object with `title`, `date` and `content`.

    Use functions `parse_title`, `parse_date` and `parse_news_content` to get individual attributes.

    Please, keep the sleep() to prevent overflow of IES website.
    '''

    time.sleep(pause)

    pass

In [ ]:
def get_all_news(link):
    '''
    wraping fuctions that accepts a `link` pointing towards hub website with links to parse and returing a dataframe containing all the links
    '''
    pass

In [ ]:
news = get_all_news('https://ies.fsv.cuni.cz/content/tree/index/lang/en')
news.head()

## Task 4: Convert Task 3 into OOP

In [ ]:
class News:
    def __init__(self,link):
        pass




class Downloader:
    def __init__(self,hub_url):
        pass


dl = Downloader('https://ies.fsv.cuni.cz/content/tree/index/lang/en')
dl.df